# [Day 17] 로봇 조련하기: 트랙바(Trackbar) 제어

드디어 로봇을 움직일 시간입니다! 하지만 바로 카메라를 연결하면 로봇이 오작동할 수 있습니다.
오늘은 **트랙바(Trackbar)**를 사용해 수동으로 모터를 움직여보며 로봇의 **한계 범위**를 안전하게 확인해보겠습니다.

## 1. 트랙바 만들기

OpenCV 창에 슬라이더를 붙일 수 있습니다.
`cv.createTrackbar(이름, 창이름, 초기값, 최대값, 콜백함수)`

In [ ]:
# 포트 선택 도구 가져오기
import serial.tools.list_ports
import ipywidgets as widgets
from IPython.display import display

# 1. 컴퓨터에 연결된 시리얼 포트 찾기
ports = serial.tools.list_ports.comports()
available_ports = [port.device for port in ports]

if not available_ports:
    print("감지된 포트가 없습니다. 임시로 COM1을 목록에 추가합니다.")
    available_ports = ['COM1']

# 2. 드롭다운 메뉴 만들기
port_dropdown = widgets.Dropdown(
    options=available_ports,
    value=available_ports[0],
    description='포트 선택:',
    disabled=False,
    )

print("아두이노가 연결된 포트를 선택해주세요:")
display(port_dropdown)

In [ ]:
import cv2 as cv
import serial
import time
import numpy as np # 빈 화면 만들기용

# 콜백 함수 (트랙바를 움직일 때마다 호출됨 - 여기선 할 일이 없어서 비워둡니다)
def nothing(x):
    pass

# 포트 설정
PORT = port_dropdown.value
try:
    ser = serial.Serial(PORT, 115200)
except:
    ser = None
    print(f"시리얼 연결 실패 ({PORT}) - 테스트 모드로 진행합니다")

# 빈 검은색 화면 생성
img = np.zeros((300, 512, 3), np.uint8)
cv.namedWindow('Robot Controller')

# 5개의 트랙바 생성 (0~180도)
cv.createTrackbar('Thumb', 'Robot Controller', 0, 180, nothing)
cv.createTrackbar('Index', 'Robot Controller', 0, 180, nothing)
cv.createTrackbar('Middle', 'Robot Controller', 0, 180, nothing)
cv.createTrackbar('Ring', 'Robot Controller', 0, 180, nothing)
cv.createTrackbar('Pinky', 'Robot Controller', 0, 180, nothing)

while True:
    cv.imshow('Robot Controller', img)
    
    # 트랙바 값 읽어오기
    th = cv.getTrackbarPos('Thumb', 'Robot Controller')
    idx = cv.getTrackbarPos('Index', 'Robot Controller')
    mid = cv.getTrackbarPos('Middle', 'Robot Controller')
    rng = cv.getTrackbarPos('Ring', 'Robot Controller')
    pnk = cv.getTrackbarPos('Pinky', 'Robot Controller')
    
    # 패킷 생성 및 전송
    # FR0[엄지]1[검지]...
    packet = f"FR0{th:03d}1{idx:03d}2{mid:03d}3{rng:03d}4{pnk:03d}
"
    
    if ser:
        ser.write(packet.encode())
        # 너무 자주 보내면 아두이노가 힘들어하므로 약간의 딜레이
        time.sleep(0.05)
    else:
        # 연결 안 됐을 땐 화면에라도 출력
        print(packet.strip())
        time.sleep(0.5)

    if cv.waitKey(1) == ord('q'):
        break

if ser: ser.close()
cv.destroyAllWindows()

## 2. 숙제: 내 로봇의 한계 찾기

슬라이더를 조금씩 움직여보며 다음 값을 찾아 메모해두세요.
1. **최소 각도 (주먹)**: 손가락이 완전히 굽혀져서 닿기 직전의 각도.
2. **최대 각도 (펴짐)**: 손가락이 쫙 펴진 각도.

**주의!** 무리하게 값을 올리면 모터가 '지잉~' 하고 떨리거나 뜨거워질 수 있습니다. 즉시 값을 낮추세요. 모터가 고장 날 수 있습니다.